# [1단계] 환경 설정 및 기본 함수 정의

## 1-1. 필수 라이브러리 설치

In [1]:
!pip install -q -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q -U transformers datasets accelerate peft trl==0.12.2 huggingface_hub
!pip install -q flash-attn --no-build-isolation
!pip install -q liger-kernel

## 1-2. 기본 라이브러리 임포트

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from huggingface_hub import login
from liger_kernel.transformers import apply_liger_kernel_to_qwen2
from google.colab import userdata

print("✅ 라이브러리 설치 및 임포트 완료!")

✅ 라이브러리 설치 및 임포트 완료!


## 1-3. 핵심 헬퍼 함수 정의

In [10]:
# [토론] Instruct 모델을 직접 사용하는 것이 안정적이라는 결론 반영
def load_model_and_tokenizer(model_id: str):
    """Instruct 모델 로드 + 안전/최적화 설정 일괄 적용."""
    try:
        apply_liger_kernel_to_qwen2()                 # 커널 패치 먼저
        print("--- Liger-Kernel 적용 완료 ---")
    except Exception as e:
        print(f"❗️ Liger-Kernel 적용 중 오류: {e}")

    model = AutoModelForCausalLM.from_pretrained(     # 모델 로드
        model_id,
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2",
        device_map="auto",
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)  # 토크나이저 1회 로드

    if tokenizer.pad_token is None:                   # pad 토큰 안전장치
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"                   # 추론 권장. 학습에도 통일 OK

    print("--- 토크나이저 로딩 및 설정 완료 ---")
    print(f"Pad token: {tokenizer.pad_token}, Padding side: {tokenizer.padding_side}")
    return model, tokenizer

# [토론] LoRA 학습 시 embed_tokens, lm_head를 함께 학습하면 성능 향상
def build_lora_config() -> LoraConfig:
    """
    순수한 LoRA-Only 설정을 생성한다.
    """
    return LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        task_type="CAUSAL_LM",
    )

def sanity_check(model, tokenizer):
    """
    토크나이저와 모델 임베딩 크기를 확인하여 설정이 올바른지 검증한다.
    resize_token_embeddings가 호출되지 않았는지 간접적으로 확인하는 역할.
    """
    print("--- Sanity Check ---")
    print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
    print(f"Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"Tokenizer length: {len(tokenizer)}")
    # Qwen2 아키텍처는 model.model.embed_tokens 경로를 사용
    print(f"Embedding size: {model.model.embed_tokens.weight.shape[0]}")
    print("--------------------")

print("✅ 핵심 헬퍼 함수 정의 완료!")

✅ 핵심 헬퍼 함수 정의 완료!


# [2단계] 파인튜닝 실행

## 2-1. 로그인 및 기본 설정

In [11]:
login(token=userdata.get('HF_TOKEN'))
HF_USERNAME = userdata.get('HF_USERNAME')
MODEL_ID = "Qwen/Qwen2.5-14B-Instruct"
DATASET_ID = f"{HF_USERNAME}/combined-dataset-30K-final-v2"
OUTPUT_DIR = "Qwen2.5-14B-KB-Finance-LoRA-v2"
FINAL_HUB_REPO_ID = f"{HF_USERNAME}/{OUTPUT_DIR}"

## 2-2. 모델, 토크나이저, 데이터셋 준비

In [12]:
model, tokenizer = load_model_and_tokenizer(MODEL_ID)
dataset = load_dataset(DATASET_ID)
sanity_check(model, tokenizer) # 준비 완료 후 최종 검증

Applied Liger kernels to Qwen2
--- Liger-Kernel 적용 완료 ---


/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1582: UserWarning: Current model requires 6272 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

## 2-3. wandb 설정

In [6]:
report_to = "none"
if userdata.get('WANDB_API_KEY'):
    report_to = "wandb"
    os.environ['WANDB_PROJECT'] = "sk-networks-ai-camp-final"
    print("✅ wandb 연동 활성화")

✅ wandb 연동 활성화


## 2-4. 훈련 계획(SFTConfig) 수립

In [13]:
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    hub_model_id=FINAL_HUB_REPO_ID,
    report_to=report_to,
    run_name=f"{OUTPUT_DIR}-run",
    num_train_epochs=2,
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    save_strategy="steps",
    save_steps=300,
    save_total_limit=2,
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=300,
    optim="paged_adamw_8bit",
    bf16=True,
    max_seq_length=2048,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    push_to_hub=True,
    hub_private_repo=True,
)

## 2-5. 트레이너(Trainer) 생성

In [8]:
# 💡 PEFT 모델을 먼저 수동으로 만들어서 device_map 충돌을 피한다.
lora_model = get_peft_model(model, build_lora_config())
print("\n--- LoRA 적용 완료 ---")
lora_model.print_trainable_parameters()

data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    instruction_template="<|im_start|>user",
    response_template="<|im_start|>assistant",
)

trainer = SFTTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    dataset_text_field="text",
    data_collator=data_collator,
)
print("\n--- SFTTrainer 준비 완료 ---")


--- LoRA 적용 완료 ---
trainable params: 1,625,948,160 || all params: 16,395,981,824 || trainable%: 9.9167


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

## 2-6. 훈련 시작

In [ ]:
print("\n--- 파인튜닝을 시작합니다 ---")
trainer.train()

## 2-7. 훈련 완료 및 결과 저장

In [ ]:
print("\n--- 파인튜닝을 시작합니다 ---")
trainer.train()

print(f"\n🎉 훈련 완료! 모델이 허깅페이스 Hub에 업로드되었습니다: {FINAL_HUB_REPO_ID}")